In [ ]:
import pandas as pd

# Extract Features
def count_packet(df):
    return len(df)

def total_length(df):
    return df["Length"].sum()

def average_packet_interval(df):
    df = df.sort_values("Time")
    return df["Time"].diff().mean()

def max_packet_interval(df):
    df = df.sort_values("Time")
    return df["Time"].diff().max()

def min_packet_interval(df):
    df = df.sort_values("Time")
    return df["Time"].diff().min()

def average_packet_length(df):
    return df["Length"].mean()

def max_packet_length(df):
    return df["Length"].max()

def min_packet_length(df):
    return df["Length"].min()

def most_common_packet_length(df):
    return df["Length"].mode().iloc[0] if not df["Length"].mode().empty else None


def get_results(df, label="ChatGPT"): # Sign Labels based on actual Website Name
    return {
        "Packet Count": count_packet(df),
        "Total Length": total_length(df),
        "Avg Interval (s)": average_packet_interval(df),
        "Max Interval (s)": max_packet_interval(df),
        "Min Interval (s)": min_packet_interval(df),
        "Avg Length (bytes)": average_packet_length(df),
        "Max Length (bytes)": max_packet_length(df),
        "Min Length (bytes)": min_packet_length(df),
        "Most Common Length (bytes)": most_common_packet_length(df),
        "Label": label
    }

# Build Files 
def process_files(file_prefix="chatGPTdata", file_range=range(1, 16)): # used actual data prefix
    total_df = pd.DataFrame()
    for i in file_range:
        df = pd.read_csv(f"{file_prefix}{i}.csv")
        results = get_results(df)
        total_df = pd.concat([total_df, pd.DataFrame([results])], ignore_index=True)
    return total_df

# Run code
if __name__ == "__main__":
    summary_df = process_files()
    print(summary_df)